
Credit: this is the code from 2captcha support team

Reference: https://2captcha.com/support/tickets/310146

In [1]:
import re
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os


In [2]:
_ = load_dotenv()

api_key = os.getenv("2CAPTCHA_API_KEY")
INVESTINGNOTE_USERNAME = os.getenv("INVESTINGNOTE_USERNAME")
INVESTINGNOTE_PASSWORD = os.getenv("INVESTINGNOTE_PASSWORD")

In [3]:
# proxy1 = "xxxx:xxxx@xx.xxx.xx.xxx:xxxx"  # YOUR PROXY
driver = webdriver.Chrome()  # You can use another WebDriver (e.g., Firefox, Edge) if needed
url = 'https://www.investingnote.com/users/sign_in'
driver.get(url)
time.sleep(5)

In [4]:
# fnd site key of InvestingNote website (or you could manually find )
src = driver.find_element(By.TAG_NAME, "iframe").get_attribute("src")
pattern = r'k=([^&]+)'
result = re.search(pattern, src)
sitekey = result.group(1)
print("sitekey: ", sitekey)

driver.find_element(By.ID, "user_login").send_keys(INVESTINGNOTE_USERNAME)
time.sleep(1)

driver.find_element(By.ID, "user_password").send_keys(INVESTINGNOTE_PASSWORD)

sitekey:  6LfAHhcTAAAAAKIc0n_1WmG3P6DA4YIojZmH7wKq


In [5]:
# REference: https://2captcha.com/p/recaptcha_v2


try:
    ## Send sitekey and pageurl to 2Captcha API.
    data = {"key": api_key,
            "method": "userrecaptcha",
            "googlekey": sitekey,
            "json": 1,
            "pageurl": url,
            # "proxy": proxy1,
            "proxytype": "HTTP",
            }
    response = requests.post(f"https://2captcha.com/in.php?", data=data)
    print("Request has been sent: ", response.text)
    s = response.json()["request"]
    time.sleep(20)

    while True:
        solu = requests.get(f"https://2captcha.com/res.php?key={api_key}&action=get&json=1&id={s}").json()
        if solu["request"] == "CAPCHA_NOT_READY":
            print(solu["request"])
            time.sleep(10)
        elif "ERROR" in solu["request"]:
            print(solu["request"])
            driver.close()
            driver.quit()
            exit(0)
        else:
            break
            
    for key, value in solu.items():
        print(key, ": ", value)

    solu = solu["request"]
    time.sleep(2)

    driver.execute_script("document.getElementsByName('g-recaptcha-response')[0].value = arguments[0];", solu)
    iframe = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/div/div/form/fieldset/section[3]/div/div/div/iframe")
    driver.switch_to.frame(iframe)
    el = driver.find_element(By.ID, "recaptcha-token")
    driver.execute_script(f"arguments[0].setAttribute('value', '{solu}');", el)
    time.sleep(1)
    driver.switch_to.default_content()

    script = """document.getElementById('sign-in-btn').removeAttribute('disabled');"""  # Making the button visible
    driver.execute_script(script)

    time.sleep(2)
    print("Answer inserted")
    time.sleep(2)
    driver.find_element(By.ID, "sign-in-btn").click()  # Click on the authorization button
    time.sleep(50)

except Exception as ex:
    print(ex)

finally:
    time.sleep(1)
    driver.close()
    driver.quit()


Request has been sent:  {"status":1,"request":"75831001538"}
status :  1
request :  03AFcWeA45mcwF3Yx7z3i3enj1e9ri2fZhrXWRPUw4y8zdUXG0hJMn4HEzjzaPSROdQfUoVSaIG-zfOBdvVbjJYTANaxna3ydGaeeELtrRBufdMBtApbZyDVYd6y4wYmkMqqRwHGvQlTrCOR9JeytOzxobWJnt-Qyh_yEllbEifIoZ35LJvlkXwE7KzEqhIQ-itns-k-SlX6_L8WyBQDNZZgE9aNV7W7XVyXWXaavC8pD1BBPe5hOm5aQsSxt0z6x6lI1kMDpB3VoPdSakCr5URYuja1OfQXUUE32Kd8933xci08i45jk06ishQoCkiJUYHESQbEP6H_RfJDK_W4RaQi5iBIfs76eAwg81ISb2f2hoAg1lkZoneh3bd44vUkQl68omoBCUeyxcJSfg1muyiph1S6bPXUsAggFR7_Tj1XnKtAyPcJEBf4DdFSG2k05M_lgMRgFwH2FOZ8vNBKdbU2sCu9s7L36bQxPYtCU75EBsSkQ5cZ5ODZKc6uDdeGdRT4C5UkMWOS-aVqQ7-0SRmdzJuaRRRyi7HQmVMlfOOspphOS4xJnxeSZgO5hY7Acc4bK7HDTmcCfuQCeEZh3YMUmsjFwERY-95-E_iJdZBULu8qtD4jNrOds_rSz0LtGG3_aubR_VIA6KSFfCbiehDw_IXssrZNVwX-KRYRyw7itbSGr6lU4DFibB8QvBO6CpfEkPHv9blg7wPda8T_qd3uwyFoUiBq8InOFjqNeQkXYJyhooYcVKSwkQxV-xllaoCZ47ROHRA84P
Answer inserted


## Computing Environment

In [6]:
%load_ext watermark

%watermark
%watermark  --iversions
%watermark -u -n -t -z

Last updated: 2024-03-01T22:44:49.148624+08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.21.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 12, GenuineIntel
CPU cores   : 8
Architecture: 64bit

re      : 2.2.1
requests: 2.31.0
selenium: 4.18.1

Last updated: Fri Mar 01 2024 22:44:49Malay Peninsula Standard Time

